In [2]:
import openai
import pandas as pd
import os
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load environment variables
OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

source_name = "data0"
sample_size = "5"

# Load your dataframe
df = pd.read_csv(f'source_data/{source_name}_sample_{sample_size}.csv')  # Replace with the actual path to your dataframe

# Set up OpenAI API key
openai.api_key = OPEN_AI_API_KEY

# Configure the Gemini API key
genai.configure(api_key=GEMINI_API_KEY)

# Define the model configuration for Gemini
generation_config = {
  "temperature": 0.5,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 50,  # Adjust as needed to fit your requirements
  "response_mime_type": "text/plain",
}

# Create the Gemini model
model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",  # Replace with the correct model name as needed
  generation_config=generation_config,
)

def get_gpt_explanation(buggy_code, fixed_code):
    """
    Function to get a brief explanation using GPT-3.5-Turbo of why the fixed code is correct compared to the buggy code.
    """
    prompt = f"""You are a senior software engineer explaining to a junior developer. 
                First, identify the main issue in the buggy code, then explain how the fixed code resolves it, focusing on clarity and brevity. 
                Keep the explanation concise, no more than 20 words.

    Buggy code:
    {buggy_code}

    Fixed code:
    {fixed_code}

    Why the fixed code is correct:"""
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that provides clear, brief explanations of why the fixed code is correct."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=50,  # Limit the output to around 20 tokens
        temperature=0.5
    )
    explanation = response['choices'][0]['message']['content'].strip()
    return explanation

def get_gemini_explanation(buggy_code, fixed_code):
    """
    Function to get a brief explanation using Gemini API of why the fixed code is correct compared to the buggy code.
    """
    prompt = f"""You are a senior software engineer explaining to a junior developer. 
                First, identify the main issue in the buggy code, then explain how the fixed code resolves it, focusing on clarity and brevity. 
                Keep the explanation concise, no more than 20 words, only need to have one sentence in the explanation.

    Buggy code:
    {buggy_code}

    Fixed code:
    {fixed_code}

    Why the fixed code is correct:"""

    # Start a chat session
    chat_session = model.start_chat(history=[])
    
    # Send the message (prompt) to the chat session
    response = chat_session.send_message(prompt)

    # Extract the text from the response
    explanation = response.text.strip()
    return explanation

def calculate_similarity(text1, text2):
    """
    Calculate cosine similarity between two pieces of text using SentenceTransformer.
    """
    # Load the pre-trained model from SentenceTransformers
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Encode the two texts
    embeddings = model.encode([text1, text2])

    # Calculate cosine similarity
    similarity_score = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
    
    return similarity_score

# Apply both functions to each row in the DataFrame and store the results in new columns
df['gpt_explanation'] = df.apply(lambda row: get_gpt_explanation(row['buggy_code'], row['fixed_code']), axis=1)
df['gemini_explanation'] = df.apply(lambda row: get_gemini_explanation(row['buggy_code'], row['fixed_code']), axis=1)

# Calculate similarity score between 'gpt_explanation' and 'gemini_explanation' for each row
df['similarity_score'] = df.apply(lambda row: calculate_similarity(row['gpt_explanation'], row['gemini_explanation']), axis=1)

# Save the dataframe with both explanations and similarity scores to a new CSV file
df.to_csv(f'output_data/bug_fix_explain_{source_name}_sample_{sample_size}_output_gpt_gemini_combined.csv', index=False)

print("Batch processing complete. Explanations and similarity scores have been generated and saved successfully.")


Batch processing complete. Explanations and similarity scores have been generated and saved successfully.
